In [776]:
import pandas as pd
import psycopg2
from psycopg2 import OperationalError

def create_connection(db_name, db_user, db_password, db_host, db_port):
    connection = None
    try:
        connection = psycopg2.connect(
            database=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port,
        )
        print("Connection to PostgreSQL DB successful")
    except OperationalError as e:
        print(f"The error '{e}' occurred")
    return connection


def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except OperationalError as e:
        print(f"The error '{e}' occurred")


connection = create_connection(
    'postgres', 'postgres', 'postgres', '5.35.13.12', '5432'
)

select_clim = "SELECT * FROM alex.climat_geo"
#INFORMATION_SCHEMA.COLUMNS where TABLE_NAME='alex.climat_geo'"
clims = execute_read_query(connection, select_clim)

#select_clim_columns = "SELECT * FROM INFORMATION_SCHEMA.COLUMNS where TABLE_NAME='alex.climat_geo'"
#select_clim_columns = "SELECT Columns FROM alex.climat_geo"
#col_names = execute_read_query(connection, select_clim_columns)


col_names = ['subject', 'city', 'lat', 'Ing', 'id']
df = pd.DataFrame(clims, columns = col_names)
df.head(5)

Connection to PostgreSQL DB successful


,subject,city,lat,Ing,id
0,Кабардино-Балкарская Республика,город Нальчик,43.485259,43.607081,2
1,Иркутская область,п Усть-Ордынский,52.805121,104.753641,3
2,Иркутская область,Тулун,54.55712,100.578047,4
3,Иркутская область,Тайшет,55.940502,98.002982,5
4,Иркутская область,п Слюдянка,57.97442,112.769605,6


In [829]:
def make_df(name:str):
    select_table = f"SELECT * FROM cbr.{name}"
    table = execute_read_query(connection,  select_table)

    cols_expr = f"SELECT column_name FROM cbr.COLUMNS where TABLE_NAME = '{name}' and table_schema = 'cbr'"
    cols = execute_read_query(connection, cols_expr)
    cols = tup_to_list(cols)

    df = pd.DataFrame(table, columns = cols)
    return df



def tup_to_list(cols):
    new_cols = []
    for col in cols:
        new_cols.append(col[0])
    return new_cols

In [250]:
# realty_unique
real_unique = make_df('realty_uniq')
real_unique.head(2)

,id,reestr_code,department,name,address,region,area,city,ya_address,lng,lat,object_id,street,climat_id
0,21,04-01-04-00-00-014,Отделение Благовещенск,Административное здание,"676776, Амурская обл., г.Райчихинск, Победы ул...",Амурская область,городской округ Райчихинск,Райчихинск,"Россия, Амурская область, Райчихинск, улица По...",129.401409,49.799777,04-01-04-00-00-014,"улица Победы, 49",None
1,404,37-06-01-00-01-037(комп.),Отделение Магадан,Склад административного здания Центрального ба...,"685000, Магаданская обл., г. Магадан,, ул. Пуш...",Магаданская область,городской округ Магадан,Магадан,"Россия, Магадан, улица Пушкина, 4",150.804651,59.566828,37-06-01-00-01-037,"улица Пушкина, 4",None


In [825]:
# expenses_date
expen = make_df('expenses_2')
expen = expen[expen['sum'] != 'Отнесено']
expen.head(2)

,year,smeta,departament,code,code_expenses,name_expenses,direction_expenses,sum_begin,sum,sum_endda,date,adress,id,object_id,lng,lat,type_expenses
0,2019.0,Хозяйственно-эксплуатационное направление ЕАО,Отделение по Еврейской автономной области Даль...,134.11,134.1100.000000,Расходы на коммунальные услуги,Оплата за электроэнергию по тарифам - ст.134.1100,None,"346388,9","346388,9",19.02.2019,,None,None,132.940799,48.786228,Расходы на коммунальные услуги
1,2019.0,Хозяйственно-эксплуатационное направление ЕАО,Отделение по Еврейской автономной области Даль...,134.11,134.1100.000000,Расходы на коммунальные услуги,Оплата за электроэнергию по тарифам - ст.134.1100,None,"326646,72","326646,72",19.03.2019,,None,None,132.940799,48.786228,Расходы на коммунальные услуги


In [827]:
print(expen.lng.nunique())
print(expen.lat.nunique())

45
83


In [779]:
expen['month'] = (expen.date.str[3:-5:1]).astype('int32')

In [ ]:
# code_exp
code_exp = make_df('code_expenses')
code_exp.head(2)

In [ ]:
# real_years
real_years = make_df('realty_list_year')
real_years.head(2)

In [195]:
expen.shape

(23156, 16)

In [206]:
expen.head(5)

,code,year,name_expenses,sum,sum_endda,date,adress
0,132.0111,2016.0,Оплата потребления электроэнергии,"136613,67","136613,67",08.08.2016,ст.
1,132.0111,2016.0,Оплата потребления электроэнергии,"157328,99","157328,99",12.09.2016,ст.
2,132.0111,2016.0,Оплата потребления электроэнергии,"154446,37","154446,37",07.10.2016,ст.
3,132.0111,2016.0,Оплата потребления электроэнергии,"145996,33","145996,33",10.11.2016,ст.
4,132.0111,2016.0,Оплата потребления электроэнергии,"150687,65","150687,65",06.12.2016,ст.


# Первая очистка

In [773]:
real_unique_cols = ['address', 'region', 'city', 'lng', 'lat']  
real_unique = real_unique[real_unique_cols]

In [780]:
expen_cols = ['type_expenses', 'month', 'name_expenses', 'sum', 'sum_endda', 'date', 'lng', 'lat', 'smeta', 'departament']     # numerical - sum и sum_endda (что-то из этого - таргет)
expen = expen[expen_cols]

In [ ]:
#from decimal import *
#expen[expen.code == Decimal('134.1')].shape

## Make Dataframe

In [255]:
real_unique.head(2)

,address,region,city,lng,lat
0,"676776, Амурская обл., г.Райчихинск, Победы ул...",Амурская область,Райчихинск,129.401409,49.799777
1,"685000, Магаданская обл., г. Магадан,, ул. Пуш...",Магаданская область,Магадан,150.804651,59.566828


In [256]:
expen.head(2)

,type_expenses,year,name_expenses,sum,sum_endda,date,adress
0,Отопление,2017.0,Оплата потребления тепловой энергии,"16805,22","16805,22",15.01.2018,ст.
1,Вода,2016.0,"Оплата водоснабжения, канализации","619,69","619,69",22.12.2016,Другие расходы.Канализация Гараж ст.


# Model

In [272]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error

def linear_regression_solution(X_tr, y_tr, X_val, y_val):

    model = LinearRegression()

    model.fit(X_tr, y_tr)
    pred = model.predict(X_val)

    score = mean_absolute_percentage_error(pred, y_val, squared = False)

    return pred, score

## One hot encoding

In [285]:
def one_hot(df, categorical:list):
    # categorical - список категориальных признаков 
    if len(categorical) == 0:
        return df
    dummies = pd.get_dummies(df[categorical], columns = categorical, drop_first = True)
    df = df.drop(categorical, axis=1)
    df = pd.concat([df, dummies], axis=1)
    return df

## prediction

In [307]:
#df_heat = one_hot(df_heat,[])

In [420]:
df = expen[['type_expenses','lng', 'lat', 'sum']]
df['lng'] = df.lng.astype('float32')
df['lat'] = df.lat.astype('float32')
df = df.rename(columns = {'sum':'target'})
num_cols = ['lng', 'lat']
df[num_cols] = df[num_cols].fillna(0)
df['target'] = (df.target.str.replace(',', '.')).astype('float32')

C:\Users\Geforce\AppData\Local\Temp\ipykernel_8968\2641035010.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lng'] = df.lng.astype('float32')
C:\Users\Geforce\AppData\Local\Temp\ipykernel_8968\2641035010.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lat'] = df.lat.astype('float32')


In [422]:
df = one_hot(df,['type_expenses'])
df

,lng,lat,target,type_expenses_Газ,type_expenses_Коммунальные услуги (расходы прошлых лет),type_expenses_Отопление,type_expenses_Расходы на коммунальные услуги,type_expenses_Электричество,type_expenses_Электроэнергия
0,0.0,0.0,33.279999,False,False,False,False,False,False
1,0.0,0.0,54.389999,False,False,False,True,False,False
2,0.0,0.0,2159.149902,False,False,False,True,False,False
3,0.0,0.0,31.250000,False,False,False,True,False,False
4,0.0,0.0,15.620000,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...
23150,0.0,0.0,63.000000,False,False,False,False,False,False
23151,0.0,0.0,1256.920044,False,False,False,False,False,False
23152,0.0,0.0,270.079987,False,False,False,False,False,False
23153,0.0,0.0,33.279999,False,False,False,False,False,False


In [423]:
df.isna().sum()

lng                                                        0
lat                                                        0
target                                                     0
type_expenses_Газ                                          0
type_expenses_Коммунальные услуги (расходы прошлых лет)    0
type_expenses_Отопление                                    0
type_expenses_Расходы на коммунальные услуги               0
type_expenses_Электричество                                0
type_expenses_Электроэнергия                               0
dtype: int64

In [308]:
#df_heat = df_heat.fillna(0)

In [309]:
#df_heat['target'] = (df_heat.target.str.replace(',', '.')).astype('float32')

In [424]:
X = df.drop(['target'], axis=1)
y = df.target
model = LinearRegression()
model.fit(X,y)
y_pred = model.predict(X)
1 - mean_absolute_percentage_error(y, y_pred)

C:\Users\Geforce\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\utils\validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\Geforce\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\utils\validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\Geforce\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\utils\validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)`

-1268.494140625

In [425]:
y_pred

array([ 3186.5156, 41592.906 , 41592.906 , ...,  3186.5156,  3186.5156,
        3186.5156], dtype=float32)

In [426]:
y

0          33.279999
1          54.389999
2        2159.149902
3          31.250000
4          15.620000
            ...     
23150      63.000000
23151    1256.920044
23152     270.079987
23153      33.279999
23154      33.279999
Name: target, Length: 23150, dtype: float32

In [319]:
y.mean()

113647.94

# Update

In [ ]:
update_columns = f"UPDATE таблица_с_тестом SET Отнесено = {}  WHERE таблица_с_тестом.type_expenses == 'Отопление'"
execute_read_query(connection, update_columns)

In [785]:
expen

,type_expenses,month,name_expenses,sum,sum_endda,date,lng,lat,smeta,departament
0,Электроэнергия,10,Оплата потребления электроэнергии,"255993,16","255993,16",17.10.2017,150.798624,59.566828,Смета Аппарата Отделения по Магаданской област...,Отделение по Магаданской области Дальневосточн...
1,Электроэнергия,11,Оплата потребления электроэнергии,"252998,45","252998,45",10.11.2017,150.798624,59.566828,Смета Аппарата Отделения по Магаданской област...,Отделение по Магаданской области Дальневосточн...
2,Электроэнергия,12,Оплата потребления электроэнергии,"247459,4","247459,4",07.12.2017,150.798624,59.566828,Смета Аппарата Отделения по Магаданской област...,Отделение по Магаданской области Дальневосточн...
3,Электроэнергия,12,Оплата потребления электроэнергии,"333551,51","333551,51",28.12.2017,150.798624,59.566828,Смета Аппарата Отделения по Магаданской област...,Отделение по Магаданской области Дальневосточн...
4,Электроэнергия,1,Оплата потребления электроэнергии,"175,62","175,62",30.01.2017,150.798624,59.568337,Смета Аппарата Отделения по Магаданской област...,Дальневосточное главное управление Центральног...
...,...,...,...,...,...,...,...,...,...,...
23150,Электроэнергия,8,Оплата потребления электроэнергии,"427047,23","427047,23",11.08.2017,150.798624,59.566828,Смета Аппарата Отделения по Магаданской област...,Дальневосточное главное управление Центральног...
23151,Электроэнергия,9,Оплата потребления электроэнергии,"-113994,23","-113994,23",01.09.2017,150.798624,59.566828,Смета Аппарата Отделения по Магаданской област...,Отделение по Магаданской области Дальневосточн...
23152,Электроэнергия,9,Оплата потребления электроэнергии,"260271,97","260271,97",01.09.2017,150.798624,59.566828,Смета Аппарата Отделения по Магаданской област...,Отделение по Магаданской области Дальневосточн...
23153,Электроэнергия,10,Оплата потребления электроэнергии,"-587890,4","-587890,4",17.10.2017,150.798624,59.566828,Смета Аппарата Отделения по Магаданской област...,Отделение по Магаданской области Дальневосточн...


In [795]:
df = expen[['type_expenses','lng', 'lat', 'sum', 'month', 'smeta', 'departament']]
df['lng'] = df.lng.astype('float32')
df['lat'] = df.lat.astype('float32')
df = df.rename(columns = {'sum':'target'})
num_cols = ['lng', 'lat']
df[num_cols] = df[num_cols].fillna(0)
df['target'] = (df.target.str.replace(',', '.')).astype('float32')

C:\Users\Geforce\AppData\Local\Temp\ipykernel_8968\74949164.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lng'] = df.lng.astype('float32')
C:\Users\Geforce\AppData\Local\Temp\ipykernel_8968\74949164.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['lat'] = df.lat.astype('float32')


In [796]:
X_ctb = df.drop('target', axis=1)
y_ctb = df.target

In [797]:
# Берем по модулю?
#y_ctb = abs(y_ctb)

In [798]:
X_ctb

,type_expenses,lng,lat,month,smeta,departament
0,Электроэнергия,150.79863,59.566830,10,Смета Аппарата Отделения по Магаданской област...,Отделение по Магаданской области Дальневосточн...
1,Электроэнергия,150.79863,59.566830,11,Смета Аппарата Отделения по Магаданской област...,Отделение по Магаданской области Дальневосточн...
2,Электроэнергия,150.79863,59.566830,12,Смета Аппарата Отделения по Магаданской област...,Отделение по Магаданской области Дальневосточн...
3,Электроэнергия,150.79863,59.566830,12,Смета Аппарата Отделения по Магаданской област...,Отделение по Магаданской области Дальневосточн...
4,Электроэнергия,150.79863,59.568336,1,Смета Аппарата Отделения по Магаданской област...,Дальневосточное главное управление Центральног...
...,...,...,...,...,...,...
23150,Электроэнергия,150.79863,59.566830,8,Смета Аппарата Отделения по Магаданской област...,Дальневосточное главное управление Центральног...
23151,Электроэнергия,150.79863,59.566830,9,Смета Аппарата Отделения по Магаданской област...,Отделение по Магаданской области Дальневосточн...
23152,Электроэнергия,150.79863,59.566830,9,Смета Аппарата Отделения по Магаданской област...,Отделение по Магаданской области Дальневосточн...
23153,Электроэнергия,150.79863,59.566830,10,Смета Аппарата Отделения по Магаданской област...,Отделение по Магаданской области Дальневосточн...


In [799]:
from sklearn.model_selection import train_test_split

X_tr, X_test, y_tr, y_test = train_test_split(X_ctb, y_ctb, random_state=42)
categorical_features = ['type_expenses']
feature_names = list(X_tr.columns)

In [816]:
## Catboost
from catboost import Pool, CatBoostRegressor
text_columns = ['smeta', 'departament']

train_pool = Pool(X_tr,
                  y_tr,
                  cat_features=categorical_features,
                  text_features=text_columns,
                  feature_names=feature_names
                  )

test_pool = Pool(X_test,
                 y_test,
                 cat_features=categorical_features,
                 text_features=text_columns,
                 feature_names=feature_names
                 )

eval_set = [(X_test, y_test)]

ctb = CatBoostRegressor(iterations=100,
                              #depth = 6,
                              loss_function='MAPE',
                              verbose=1,
                              random_state=42,
                              eval_metric='MAPE',
                              #nan_mode='Min',
                              ) 

ctb.fit(train_pool,
              verbose=1,
              early_stopping_rounds=20,
              eval_set=eval_set
              )

y_pred = ctb.predict(X_test)

0:	learn: 0.9898386	test: 0.9907696	best: 0.9907696 (0)	total: 37.5ms	remaining: 3.72s
1:	learn: 0.9895244	test: 0.9905137	best: 0.9905137 (1)	total: 82ms	remaining: 4.02s
2:	learn: 0.9887879	test: 0.9899252	best: 0.9899252 (2)	total: 131ms	remaining: 4.24s
3:	learn: 0.9884858	test: 0.9896196	best: 0.9896196 (3)	total: 176ms	remaining: 4.22s
4:	learn: 0.9882203	test: 0.9894482	best: 0.9894482 (4)	total: 224ms	remaining: 4.25s
5:	learn: 0.9878453	test: 0.9892338	best: 0.9892338 (5)	total: 271ms	remaining: 4.25s
6:	learn: 0.9869244	test: 0.9887571	best: 0.9887571 (6)	total: 317ms	remaining: 4.21s
7:	learn: 0.9863392	test: 0.9883629	best: 0.9883629 (7)	total: 364ms	remaining: 4.18s
8:	learn: 0.9856204	test: 0.9879478	best: 0.9879478 (8)	total: 410ms	remaining: 4.15s
9:	learn: 0.9854636	test: 0.9877970	best: 0.9877970 (9)	total: 457ms	remaining: 4.11s
10:	learn: 0.9853098	test: 0.9877601	best: 0.9877601 (10)	total: 506ms	remaining: 4.09s
11:	learn: 0.9847670	test: 0.9874573	best: 0.9874573

In [814]:
mean_absolute_percentage_error(y_test, y_pred)

1.0537343246853015

In [815]:
mean_absolute_percentage_error(y_test, [y_test.mean()] * y_test.shape[0])

1374.7008

In [824]:
# метрика, если ответ - нули
mean_absolute_percentage_error(y_test, [0] * y_test.shape[0])

1.0

In [817]:
feature_imp = pd.Series(ctb.get_feature_importance(),
                     X_tr.columns)
feature_imp

type_expenses     0.000000
lng               0.011753
lat               0.251398
month             3.315395
smeta            87.253593
departament       9.167861
dtype: float64

In [691]:
mean_absolute_percentage_error([1,1,1], [2,2,2])

1.0

In [710]:
mean_absolute_percentage_error([100000,100000,100000], [1,1,1])

0.99999

In [674]:
mean_absolute_percentage_error([1], [100000])

99999.0